In [ ]:
import pandas as pd
import numpy as np
import ydata_profiling as pp

import math
import datetime as dt

import matplotlib.pyplot as plt
from itertools import cycle
import plotly.express as px
from plotly.subplots import make_subplots
%matplotlib inline

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout , Input, LSTM


In [ ]:
data=pd.read_csv('BTC.csv')

In [ ]:
pp.ProfileReport(data)

In [ ]:
print('Total number of days present in the dataset: ',data.shape[0])
print('Total number of fields present in the dataset: ',data.shape[1])

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.info()

In [ ]:
data.describe

In [ ]:
print('Null Values: ',data.isnull().values.sum())

In [ ]:
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data['Date']

# Overall Analysis

In [ ]:
y_overall = data.loc[(data['Date'] >= '2019-05-31')
                     & (data['Date'] <= '2024-05-31')]

y_overall.drop(y_overall[['Adj Close','Volume']],axis=1)

In [ ]:
names = cycle(['Open','Close','High','Low'])

fig = px.line(y_overall, x=y_overall.Date, y=[y_overall['Open'], y_overall['Close'], 
                                          y_overall['High'], y_overall['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Visual Representation of Data', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

In [ ]:
monthvise = y_overall.groupby(y_overall['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

In [ ]:
data=data[['Date','Close']]
data

In [ ]:
fig = px.line(data, x=data.Date, y=data.Close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Whole period of timeframe of Bitcoin close price 2019-2024', plot_bgcolor='white', 
                  font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

# Now we will Take data of just 1 Year

In [ ]:
data = data[(data['Date'] > '2021-02-19') & (data['Date'] <= '2022-02-19')]
close_stock = data.copy()
data

In [ ]:
print("Total data for prediction: ",data.shape[0])

# Train Model

In [ ]:
del data['Date']
scaler=MinMaxScaler(feature_range=(0,1))
data=scaler.fit_transform(np.array(data).reshape(-1,1))
print(data.shape)

In [ ]:
training_size=int(len(data)*0.67)
test_size=len(data)-training_size
train_data,test_data=data[0:training_size,:],data[training_size:len(data),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

In [ ]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

In [ ]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

# LSTM

In [ ]:
model=Sequential()

model.add(LSTM(10,input_shape=(None,1),activation="relu"))

model.add(Dense(1))

model.compile(loss="mean_squared_error",optimizer="adam")

In [ ]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

# Plotting Loss vs Validation loss 

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

# Model Evaluation

In [ ]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

# Evaluation metrices RMSE, MSE and MAE

In [ ]:
print(f'Train data RMSE: {math.sqrt(mean_squared_error(original_ytrain,train_predict)):.2f}')
print(f'Train data MSE: { mean_squared_error(original_ytrain,train_predict):.2f}')
print(f'Train data MAE: {mean_absolute_error(original_ytrain,train_predict):.2f}')
print("--"* 50)
print(f'Test data RMSE: {math.sqrt(mean_squared_error(original_ytest,test_predict)):.2f}')
print(f'Test data MSE: {mean_squared_error(original_ytest,test_predict):.2f}')
print(f'Test data MAE: {mean_absolute_error(original_ytest,test_predict):.2f}')

# Variance Regression Score

In [ ]:
print(f'Train data explained variance regression score: {explained_variance_score(original_ytrain, train_predict):.2f}')
print(f'Test data explained variance regression score: {explained_variance_score(original_ytest, test_predict):.2f}')

# R square score for regression

In [ ]:
print(f'Train data R2 score: {(r2_score(original_ytrain, train_predict)):.2f}')
print(f'Test data R2 score: {(r2_score(original_ytest, test_predict)):.2f}')

# Comparision of original stock close price and predicted close price

In [ ]:
look_back=time_step
trainPredictPlot = np.empty_like(data)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict

testPredictPlot = np.empty_like(data)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(data)-1, :] = test_predict

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Year'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()